In [ ]:
%matplotlib inline
import fceulib
from fceulib import VectorBytes
import numpy as np
import time
import matplotlib.pyplot as plt
from fceu_help import *
from math import *

In [ ]:

START = 0x08
RUN = 0x02
JUMP = 0x01

#inputVec = fceulib.readInputs('videos/algar-batman.fm2')
#emu = fceulib.runGame('batman.nes')

#inputVec = fceulib.readInputs('videos/lordtom_tompa-smb3-warpless.fm2')
#emu = fceulib.runGame('smb3.nes')

inputVec = fceulib.readInputs('videos/happylee4-smb-warpless.fm2')
emu = fceulib.runGame('mario.nes')

#inputVec = fceulib.readInputs('videos/Contra by 3D.fm2')
#emu = fceulib.runGame('contra.nes')

start = VectorBytes()
emu.save(start)



In [ ]:
peekevery = 1
emu.load(start)
image_prefix = 'contra'
display = False
img_buffer = VectorBytes()

colorized2id = {}
id2colorized = {}
with open('test.txt','wb') as outfile:
    for timestep, inp in enumerate(inputVec):
        emu.stepFull(inp,0x0)
        if not (timestep % peekevery == 0) or timestep < 270 :
            continue
        print timestep
        if timestep > 1000:
            break

        sprite_list,colorized_sprites = get_all_sprites(emu.fc)
        if display:
            outputImage(emu, '{}_{}'.format(image_prefix,timestep),img_buffer)
            #if len(sprite_list) > 0:
            #    for ii in range(len(sprite_list)):
            #        print sprite_list[ii][0],sprite_list[ii][1]
            #        plt.imshow((colorized_sprites[ii][:,:,:3]*1.0)/255.0,interpolation='nearest')
            #        plt.show()
            if len(sprite_list) > 0:
                sprite_list2 = np.array(sprite_list,dtype='float')
                print sprite_list2
                plt.plot(sprite_list2[:,0]/8.,sprite_list2[:,1]/8.,'rx') 
            nt = pointer_to_numpy(emu.fc.ppu.NTARAM)
            nt = nt[:960].reshape(30,32)
            plt.imshow(nt,interpolation='none')
            plt.show()
        
        for sprite_id,sprite in enumerate(sprite_list):
            if np.sum(colorized_sprites[sprite_id].ravel()) == 0:
                continue
            uniq = tuple(colorized_sprites[sprite_id].ravel())
            if uniq not in colorized2id:
                colorized2id[uniq] = len(colorized2id)
                id2colorized[colorized2id[uniq]] = colorized_sprites[sprite_id]
            outfile.write('{},{},{},{}\n'.format(timestep,colorized2id[uniq],sprite[0],sprite[1]))
        if False:
            ram_ptr = pointer_to_numpy(emu.fc.fceu.RAM)
            for ram_index,d in enumerate(ram_ptr):
                for sprite_id,sprite in enumerate(sprite_list):
                    if abs(d-sprite[0]) <= 1:
                        outfile.write('{},{},{},{}\n'.format(ram_index,'x',sprite[2],sprite_id))
                    if abs(d-sprite[1]) <= 1:
                        outfile.write('{},{},{},{}\n'.format(ram_index,'y',sprite[2],sprite_id))
           
    

In [ ]:
help(fceulib)

In [ ]:
with open('test.txt') as infile:
    data = []
    for line in infile:
        data.append([int(float(d)) for d in line.rstrip().split(',')])


In [ ]:
timesteps = {}
for d in data:
    if d[0] not in timesteps:
        timesteps[d[0]] = []
    timesteps[d[0]].append(d[1:])


In [ ]:
all_pairs = {}
spr = get_sprite(0,emu.fc)
height = spr.shape[0]

for timestep in sorted(timesteps):
    dat = timesteps[timestep]
    for ii in range(len(dat)):
        for jj in range(len(dat)):
            if ii != jj:
                id1 = dat[ii][0]
                id2 = dat[jj][0]
                dx = dat[ii][1]-dat[jj][1]
                dy = dat[ii][2]-dat[jj][2]
                #if dx % 8 != 0 or dy % 8 != 0:
                #    continue
                if abs(dx) > 8 or abs(dy) > height:
                    continue
                if id1 > id2:
                    continue
                entry = (id1,id2) 
                if entry not in all_pairs:
                    all_pairs[entry] = {}
                diff = (dx,dy)
                if diff not in all_pairs[entry]:
                    all_pairs[entry][diff] = 0
                all_pairs[entry][diff] +=1
                if id1 == id2:
                    diff = (-dx,-dy)
                    if diff not in all_pairs[entry]:
                        all_pairs[entry][diff] = 0
                    all_pairs[entry][diff] +=1

all_sprites = {}

for pair in all_pairs:
    id1 = pair[0]
    id2 = pair[1]
    if id1 not in all_sprites:
        all_sprites[id1] = {}
    if id2 not in all_sprites[id1]:
        all_sprites[id1][id2] = 0
    
    for dat in all_pairs[pair]:
        all_sprites[id1][id2] += all_pairs[pair][dat]
    id1 = pair[1]
    id2 = pair[0]
    if id1 not in all_sprites:
        all_sprites[id1] = {}
    if id2 not in all_sprites[id1]:
        all_sprites[id1][id2] = 0
    
    for dat in all_pairs[pair]:
        all_sprites[id1][id2] += all_pairs[pair][dat]

        #print pair,dat,all_pairs[pair][dat]
    #print '\n'
display = True
accepted = {}
for sprite in all_sprites:
    total = 0
    dat = []
    for other in all_sprites[sprite]:
        if False:
            plt.imshow((id2colorized[other][:,:,:3]*1.0)/255.0,interpolation='nearest')
            plt.show()
            print sprite,other,all_sprites[sprite][other]
            print '\n'
        dat.append(all_sprites[sprite][other])
    total = np.max([1,np.mean(dat)/16.0])
    for other in all_sprites[sprite]:
        if all_sprites[sprite][other] > total:
            s = sprite
            o = other
            if sprite > other:
                o = sprite
                s = other
            accepted[(s,o)] = all_sprites[sprite][other] 
for sprite,other in accepted:
    print 'PAIR ', accepted[(sprite,other)]
    plt.imshow((id2colorized[sprite][:,:,:3]*1.0)/255.0,interpolation='nearest')
    plt.show()
    plt.imshow((id2colorized[other][:,:,:3]*1.0)/255.0,interpolation='nearest')
    plt.show()
            